In [1]:
import os
os.chdir("../")

from wbe_odm import odm
from wbe_odm.odm_mappers import excel_template_mapper, mcgill_mapper, csv_mapper

import pandas as pd
import plotly.express as px
import json

In [7]:
PLOTLY_COLORS = px.colors.qualitative.Plotly

COLORS = {
    "Very High": "#c13525",
    "High": "#ff8652",
    "Medium": "#ffbb43",
    "Low": "#b6e9d1",
    "Very Low": "#6da06f",
}

DATA_FOLDER = "/Users/jeandavidt/OneDrive - Université Laval/COVID/Latest Data"
CSV_FOLDER = "/Users/jeandavidt/OneDrive - Université Laval/COVID/Latest Data/odm_csv"
QC_STATIC_DATA = os.path.join(DATA_FOLDER, "Ville de Quebec - All data - v1.1.xlsx")
QC_LAB_DATA = os.path.join(DATA_FOLDER, "CentrEau-COVID_Resultats_Quebec_final.xlsx")
QC_SHEET_NAME =  "QC Data Daily Samples (McGill)"

MTL_STATIC_DATA = os.path.join(DATA_FOLDER, "mcgill_static.xlsx")
MTL_LAB_DATA = os.path.join(DATA_FOLDER, "CentrEau-COVID_Resultats_Montreal_final.xlsx")
MTL_POLY_SHEET_NAME = "Mtl Data Daily Samples (Poly)"
MTL_MCGILL_SHEET_NAME = "Mtl Data Daily Samples (McGill)"


In [8]:
RELOAD = False
store = odm.Odm()
if RELOAD:
    qc_lab = mcgill_mapper.McGillMapper()
    mcgill_lab = mcgill_mapper.McGillMapper()
    poly_lab = mcgill_mapper.McGillMapper()

    qc_lab.read(QC_LAB_DATA, QC_STATIC_DATA, QC_SHEET_NAME, "frigon_lab")

    mcgill_lab.read(MTL_LAB_DATA, MTL_STATIC_DATA, MTL_MCGILL_SHEET_NAME, "frigon_lab")

    poly_lab.read(MTL_LAB_DATA, MTL_STATIC_DATA, MTL_POLY_SHEET_NAME, "dorner_lab")

    store.append_from(qc_lab)
    store.append_from(mcgill_lab)
    store.append_from(poly_lab)
else:
    from_csv = csv_mapper.CsvMapper()
    from_csv.read(CSV_FOLDER)
    store.append_from(from_csv)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jeandavidt/OneDrive - Université Laval /COVID/Latest Data/odm_csv'

In [4]:
store.to_csv(DATA_FOLDER, "test")

In [ ]:
one_table = store.combine_per_sample()

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [3]:
polygons = store.polygon
polygons['color'] = "grey"
for i in range(len(polygons)):
    polygons.loc[i,"color"] = COLORS[i%len(COLORS)]
for col in ["pop", "link"]:
    if col in polygons.columns:
        polygons.drop(columns=[col], inplace=True)
polys = store.get_geoJSON()
with open("polygons.geojson", "w") as f:
    f.write(json.dumps(polys))

In [6]:
def get_site_geoJSON(odm_data):
    def make_point_feature(row):
        props_to_add = ["siteID", "name", "description", "clean_type", "polygonID"]
        return {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [row["geoLong"], row["geoLat"]],
                },
            "properties": {
                k: row[k] for k in props_to_add
            }
        }
    sites = odm_data.site
    site_types = {
        "wwtpmuc": "Station de traitement des eaux usées municipale pour égouts combinés",
        "pstat": "Station de pompage",
        "ltcf": "Établissement de soins de longue durée",
        "airpln":"Avion",
        "corFcil": "Prison",
        "school": "École",
        "hosptl": "Hôpital",
        "shelter": "Refuge",
        "swgTrck": "Camion de vidange",
        "uCampus": "Campus universitaire",
        "mSwrPpl": "Collecteur d'égouts",
        "holdTnk": "Bassin de stockage",
        "retPond": "Bassin de rétention",
        "wwtpMuS": "Station de traitement des eaux usées municipales pour égouts sanitaires seulement",
        "wwtpInd": "Station de traitement des eaux usées industrielle",
        "lagoon": "Système de lagunage pour traitement des eaux usées",
        "septTnk": "Fosse septique.",
        "river": "Rivière",
        "lake": "Lac",
        "estuary": "Estuaire",
        "sea": "Mer",
        "ocean": "Océan",
    }
    sites = odm_data.site
    sites["clean_type"] = sites["type"].str.lower().map(site_types)
    sites["clean_type"]
    sites["features"] = sites.apply(lambda row: make_point_feature(row), axis=1)
    point_list = list(sites["features"])
    js = {
        "type": "FeatureCollection",
        "features": point_list
    }
    return js


In [34]:

dico_points = get_site_geoJSON(store)
with open("sites.geojson", "w") as f:
    f.write(json.dumps(dico_points))